In [1]:
import my_streamlit
import requests
import json
from utils.retriever_pipeline import retrieve_documents, retrieve_documents_by_query
from utils.doc_handler import process_documents
from sentence_transformers import CrossEncoder
import torch
import os
from dotenv import load_dotenv, find_dotenv
torch.classes.__path__ = [os.path.join(torch.__path__[0], torch.classes.__file__)]  # Fix for torch classes not found error


/workspace/DeepSeek-RAG-Chatbot/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
st = my_streamlit.streamlit()

In [3]:
for home_dir, dir, files in os.walk("./recipes"):
    break
files = [home_dir + "/" + f for f in files]

In [4]:
#load_dotenv(find_dotenv())  # Loads .env file contents into the application based on key-value pairs defined therein, making them accessible via 'os' module functions like os.getenv().

OLLAMA_BASE_URL = os.getenv("OLLAMA_API_URL", "http://localhost:11434")
OLLAMA_API_URL = f"{OLLAMA_BASE_URL}/api/generate"
MODEL= os.getenv("MODEL", "mistral")                                                      #Make sure you have it installed in ollama
MODEL = "mistral"
EMBEDDINGS_MODEL = "nomic-embed-text:latest"
CROSS_ENCODER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"

device = "cuda" if torch.cuda.is_available() else "cpu"

reranker = None                                                        # 🚀 Initialize Cross-Encoder (Reranker) at the global level 
try:
    reranker = CrossEncoder(CROSS_ENCODER_MODEL, device=device)
except Exception as e:
    print(f"Failed to load CrossEncoder model: {str(e)}")



st.session_state.messages = []
st.session_state.retrieval_pipeline = None
st.session_state.rag_enabled = False
st.session_state.documents_loaded = False

uploaded_files = files
if uploaded_files and not st.session_state.documents_loaded:
    process_documents(uploaded_files,reranker,EMBEDDINGS_MODEL, OLLAMA_BASE_URL, st)


🔗 Total Nodes: 355
🔗 Total Edges: 593
🔗 Sample Nodes: ['Салат', 'Тушки', 'Опустите', 'Затем', 'Варите', 'Нарежьте', 'Яблоки', 'Снимите', 'Достаньте', 'Переложите']
🔗 Sample Edges: [('Салат', 'Тушки'), ('Салат', 'Подготовим'), ('Салат', 'Закатываем'), ('Салат', 'Храним'), ('Салат', 'Гнездо'), ('Салат', 'Подготовьте'), ('Тушки', 'Опустите'), ('Опустите', 'Затем'), ('Затем', 'Варите'), ('Затем', 'Нарежьте')]


In [5]:
# Display messages
for message in st.session_state.messages:
    print(message["content"])

In [6]:
def make_system_prompt(chat_history, context, prompt):
     res = f"""Используйте историю чата для сохранения контекста:
     История чата:
     {chat_history}

     Проанализируйте вопрос и контекст, выполнив следующие шаги:
     1. Определите ключевые сущности и отношения
     2. Проверьте наличие противоречий между источниками.
     3. Синтезируйте информацию из нескольких контекстов
     4. Сформулируйте структурированный ответ

    Окружение:
    {context}

    Вопрос: {prompt}
    Ответ:"""
    
     return res

In [7]:
def make_system_prompt(chat_history, context, prompt):
     res = f"""

     Представь, что ты шеф-повар. 
     Напиши подробный рецепт на основе запроса: {prompt} .
     
     Используй рецепты из контекста и историю чата.
     Контекст: {context}
     История чата: {chat_history}    

     Ответ:"""
    
     return res

In [12]:
st.session_state.rag_enabled = True
st.session_state.enable_hyde = True
st.session_state.enable_reranking = True
st.session_state.enable_graph_rag = True
st.session_state.temperature = 0.3
st.session_state.max_contexts = 3

prompt = "Я не люблю картошку с сахаром?"

In [13]:
chat_history = "\n".join([msg["content"] for msg in st.session_state.messages[-5:]])  # Last 5 messages
st.session_state.messages.append({"role": "user", "content": prompt})
print(prompt)

full_response = ""

# 🚀 Build context
context = ""
if st.session_state.rag_enabled and st.session_state.retrieval_pipeline:
    try:
        docs = retrieve_documents_by_query(prompt, OLLAMA_API_URL, MODEL, chat_history, st)
        context = "\n".join(
            f"[Source {i+1}]: {doc.page_content}" 
            for i, doc in enumerate(docs)
        )
    except Exception as e:
        st.error(f"Retrieval error: {str(e)}")

print(f"\n\nContext: {context}\n\n")
# 🚀 Structured Prompt
system_prompt = make_system_prompt(chat_history, context, prompt)

# Stream response
response = requests.post(
    OLLAMA_API_URL,
    json={
        "model": MODEL,
        "prompt": system_prompt,
        "stream": True,
        "options": {
            "temperature": st.session_state.temperature,  # Use dynamic user-selected value
            "num_ctx": 4096
        }
    },
    stream=True
)
try:
    for line in response.iter_lines():
        if line:
            data = json.loads(line.decode())
            token = data.get("response", "")
            full_response += token
            if data.get("done", False):
                break
                
    print(f"\n\n{full_response}")
    st.session_state.messages.append({"role": "assistant", "content": full_response})
    
except Exception as e:
    print(f"Generation error: {str(e)}")
    st.session_state.messages.append({"role": "assistant", "content": "Извините, произошла ошибка."})

Я не люблю картошку с сахаром?


Retrieved documents using BM25 + FAISS: [Document(id='c4211056-07eb-48a6-89d5-1461e2ccb1e2', metadata={'source': './recipes/recipe_povar_49.txt'}, page_content='Хачапури с сыром на сковороде\nПриготовьте ингредиенты.\nСмешиваем сахар (3 ч. ложки), немного соли и соды, добавляем в закваску, хорошо перемешиваем. Вливаем растительного масла (3 ст.ложки), потом начинаем добавлять муку и замешивать тесто. Тесто должно подойти.\nНачинка: смешиваем сыр, яйцо, солим.\nРазминаем начинку, добавляем масло.\nРаскатываем лепешки, выкладываем начинку.\nЗалепите хачапури конвертом.\nСмажьте сковороду маслом, выложите хачапури.\nПереворачиваем, жарим до готовности, минут 15 получается. Подаем, смазав маслом. Приятного аппетита)'), Document(id='1017843c-588e-4d44-8272-7e643b194a32', metadata={'source': './recipes/recipe_povar_20.txt'}, page_content='Рыбный салат из консервов с картошкой\nПодготовьте все ингредиенты.\nКартофель очистите и нарежьте кубиками, причем можно 